In [ ]:
# Google Colab에서 실행하는 RAG 및 파인튜닝 코드

# ============================
# 1. 필요한 라이브러리 설치
# ============================
!pip install -q langchain langchain-community
!pip install -q sentence-transformers
!pip install -q chromadb
!pip install -q transformers accelerate bitsandbytes
!pip install -q peft datasets
!pip install -q pypdf
!pip install -q torch


In [8]:
import os
import torch
import warnings
warnings.filterwarnings('ignore')


# WandB 비활성화 (API 키 불필요)
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader # Corrected import
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from datasets import Dataset
import json

# ============================
# 2. 문서 로드 및 전처리
# ============================

def load_and_process_document(file_path):
    """PDF 문서를 로드하고 청크로 분할"""

    # PDF 로더 사용
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    # 텍스트 분할기 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len,
        separators=["\n\n", "\n", ".", " ", ""]
    )

    # 문서를 청크로 분할
    chunks = text_splitter.split_documents(documents)

    print(f"문서를 {len(chunks)}개의 청크로 분할했습니다.")
    return chunks

# ============================
# 3. 벡터 데이터베이스 구축
# ============================

def create_vector_database(chunks):
    """문서 청크를 임베딩하고 벡터 DB 생성"""

    # 한국어 지원 임베딩 모델 사용
    embeddings = HuggingFaceEmbeddings(
        model_name="jhgan/ko-sroberta-multitask",
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

    # ChromaDB 벡터 스토어 생성
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )

    print("벡터 데이터베이스가 생성되었습니다.")
    return vectorstore

# ============================
# 4. RAG 시스템 구현
# ============================

class SimpleRAG:
    def __init__(self, vectorstore, model_name="beomi/KoAlpaca-Polyglot-5.8B"):
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )

        # 4비트 양자화 설정
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True
        )

        # 모델 로드 (메모리 효율적)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True
        )

        # 패딩 토큰 설정
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def retrieve_context(self, query):
        """질문과 관련된 문서 검색"""
        docs = self.retriever.get_relevant_documents(query)
        context = "\n".join([doc.page_content for doc in docs])
        return context

    def generate_answer(self, query, max_length=512):
        """컨텍스트를 활용한 답변 생성"""
        # 관련 문서 검색
        context = self.retrieve_context(query)

        # 프롬프트 구성
        prompt = f"""다음 문서를 참고하여 질문에 답변해주세요.

문서:
{context}

질문: {query}

답변:"""

        # 토크나이징
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1024
        ).to(self.model.device)

        # 답변 생성
        with torch.no_grad():
            # Filter out 'token_type_ids' if present
            inputs_dict = {k: v for k, v in inputs.items() if k != 'token_type_ids'}
            outputs = self.model.generate(
                **inputs_dict,
                max_new_tokens=max_length,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.2
            )

        # 디코딩
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = answer.split("답변:")[-1].strip()

        return answer

# ============================
# 5. 파인튜닝 데이터셋 준비
# ============================

def prepare_finetuning_dataset(chunks):
    """문서 청크를 파인튜닝용 데이터셋으로 변환"""

    dataset_list = []

    for chunk in chunks[:100]:  # 메모리 제약으로 100개만 사용
        text = chunk.page_content

        # 간단한 Q&A 형식으로 변환
        if len(text) > 50:
            # 질문 생성 (간단한 예시)
            question = "다음 내용을 요약해주세요."
            answer = text[:200] if len(text) > 200 else text

            dataset_list.append({
                "instruction": question,
                "input": text,
                "output": answer
            })

    # Dataset 객체로 변환
    dataset = Dataset.from_list(dataset_list)

    print(f"파인튜닝 데이터셋 크기: {len(dataset)}")
    return dataset

# ============================
# 6. LoRA 파인튜닝
# ============================

def finetune_with_lora(model_name="beomi/KoAlpaca-Polyglot-5.8B", dataset=None):
    """LoRA를 사용한 효율적인 파인튜닝"""

    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    # 모델과 토크나이저 로드
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # LoRA 설정 - Adjusted target_modules for KoAlpaca
    lora_config = LoraConfig(
        r=8,  # LoRA rank
        lora_alpha=32,
        target_modules=["query_key_value"], # Corrected target modules
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    # LoRA 적용
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # 데이터 전처리 함수
    def preprocess_function(examples):
        prompts = []
        for inst, inp, out in zip(examples["instruction"], examples["input"], examples["output"]):
            prompt = f"### 질문: {inst}\n### 입력: {inp}\n### 답변: {out}"
            prompts.append(prompt)

        model_inputs = tokenizer(
            prompts,
            max_length=512,
            truncation=True,
            padding=True
        )
        model_inputs["labels"] = model_inputs["input_ids"].copy()
        return model_inputs

    # 데이터셋 전처리
    tokenized_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset.column_names
    )

    # 학습 설정
    training_args = TrainingArguments(
        output_dir="./lora_model",
        num_train_epochs=3,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        logging_steps=50,
        save_strategy="epoch",
        # Removed evaluation_strategy="no"
        learning_rate=2e-4,
        fp16=True,
        push_to_hub=False,
    )

    # 데이터 콜레이터
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # 트레이너 생성
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
    )

    # 학습 시작
    print("파인튜닝을 시작합니다...")
    trainer.train()

    # 모델 저장
    model.save_pretrained("./lora_model")
    tokenizer.save_pretrained("./lora_model")

    print("파인튜닝이 완료되었습니다!")
    return model, tokenizer

# ============================
# 7. 실행 코드
# ============================

def main():
    # 파일 경로 설정 (Colab에 업로드한 PDF 파일 경로)
    # 먼저 파일을 업로드하세요:
    from google.colab import files
    uploaded = files.upload()

    # 업로드된 파일 이름 확인
    file_path = list(uploaded.keys())[0]
    print(f"업로드된 파일: {file_path}")

    # 1. 문서 로드 및 처리
    print("\n[1/5] 문서를 로드하고 처리합니다...")
    chunks = load_and_process_document(file_path)

    # 2. 벡터 데이터베이스 생성
    print("\n[2/5] 벡터 데이터베이스를 생성합니다...")
    vectorstore = create_vector_database(chunks)

    # 3. RAG 시스템 초기화
    print("\n[3/5] RAG 시스템을 초기화합니다...")
    rag_system = SimpleRAG(vectorstore)

    # 4. RAG 테스트
    print("\n[4/5] RAG 시스템을 테스트합니다...")
    test_queries = [
        "AI 산업의 최신 동향은 무엇인가요?",
        "오픈AI의 최근 소식은?",
        "구글 딥마인드의 연구 내용은?"
    ]

    for query in test_queries:
        print(f"\n질문: {query}")
        answer = rag_system.generate_answer(query)
        print(f"답변: {answer[:300]}...")  # 답변 일부만 출력

    # 5. 파인튜닝 (선택사항 - 메모리가 충분한 경우)
    print("\n[5/5] 파인튜닝을 준비합니다...")
    dataset = prepare_finetuning_dataset(chunks)

    # 파인튜닝 실행 (주의: 시간이 오래 걸릴 수 있음)
    user_input = input("\n파인튜닝을 진행하시겠습니까? (y/n): ")
    if user_input.lower() == 'y':
        model, tokenizer = finetune_with_lora(dataset=dataset)
        print("\n파인튜닝이 완료되었습니다!")
    else:
        print("\n파인튜닝을 건너뜁니다.")

    print("\n모든 작업이 완료되었습니다!")
    return rag_system

# 실행
if __name__ == "__main__":
    rag_system = main()

Saving SPRi AI Brief_8월호_산업동향_F.pdf to SPRi AI Brief_8월호_산업동향_F (4).pdf
업로드된 파일: SPRi AI Brief_8월호_산업동향_F (4).pdf

[1/5] 문서를 로드하고 처리합니다...
문서를 127개의 청크로 분할했습니다.

[2/5] 벡터 데이터베이스를 생성합니다...
벡터 데이터베이스가 생성되었습니다.

[3/5] RAG 시스템을 초기화합니다...


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[4/5] RAG 시스템을 테스트합니다...

질문: AI 산업의 최신 동향은 무엇인가요?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


파인튜닝을 시작합니다...


Step,Training Loss
50,3.205500


파인튜닝이 완료되었습니다!

파인튜닝이 완료되었습니다!

모든 작업이 완료되었습니다!


In [9]:
# ====================================
# 이전 셀에서 학습된 RAG 시스템을 활용한 Gradio 인터페이스
# ====================================

# Gradio 설치 (이미 설치되어 있으면 건너뜀)
!pip install -q gradio

import gradio as gr
import torch
from datetime import datetime
import json

# ====================================
# 1. 이전 셀의 rag_system 확인 및 래퍼 클래스 생성
# ====================================

# 이전 셀에서 생성된 rag_system이 있는지 확인
try:
    if 'rag_system' not in globals():
        print("⚠️ rag_system이 없습니다. 새로 생성합니다...")

        # 기존 벡터 DB가 있으면 로드
        if os.path.exists("./chroma_db"):
            vectorstore = Chroma(
                persist_directory="./chroma_db",
                embedding_function=HuggingFaceEmbeddings(
                    model_name="jhgan/ko-sroberta-multitask",
                    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
                )
            )
            rag_system = SimpleRAG(vectorstore)
            print("✅ 기존 벡터 DB를 사용하여 RAG 시스템을 재구성했습니다.")
        else:
            print("❌ 벡터 DB가 없습니다. 먼저 문서를 처리해주세요.")
    else:
        print("✅ 기존 rag_system을 사용합니다.")
except Exception as e:
    print(f"⚠️ 시스템 확인 중 오류: {e}")

# ====================================
# 2. Gradio용 인터페이스 함수들
# ====================================

# 대화 기록 저장용
conversation_history = []

def process_question(question, temperature=0.7, max_tokens=512, use_context=True):
    """사용자 질문을 처리하고 답변 생성"""

    if not question.strip():
        return "질문을 입력해주세요.", "", ""

    try:
        # 시작 시간 기록
        start_time = datetime.now()

        # 컨텍스트 검색
        context = ""
        if use_context and hasattr(rag_system, 'retrieve_context'):
            context = rag_system.retrieve_context(question)
            context_display = f"📚 **참고된 문서:**\n```\n{context[:500]}{'...' if len(context) > 500 else ''}\n```"
        else:
            context_display = "컨텍스트를 사용하지 않았습니다."

        # 답변 생성
        answer = rag_system.generate_answer(
            question,
            max_length=max_tokens
        )

        # 처리 시간 계산
        process_time = (datetime.now() - start_time).total_seconds()

        # 대화 기록 저장
        conversation_history.append({
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'question': question,
            'answer': answer,
            'process_time': process_time
        })

        # 결과 포맷팅
        answer_display = f"""
## 💡 답변

{answer}

---
*처리 시간: {process_time:.2f}초*
"""

        # 통계 정보
        stats = f"""
- 총 대화 수: {len(conversation_history)}
- 평균 처리 시간: {sum(h['process_time'] for h in conversation_history) / len(conversation_history):.2f}초
- 현재 세션 시작: {conversation_history[0]['timestamp'] if conversation_history else 'N/A'}
"""

        return answer_display, context_display, stats

    except Exception as e:
        error_msg = f"❌ 오류 발생: {str(e)}"
        return error_msg, "", ""

def get_sample_questions():
    """예시 질문 목록 반환"""
    return [
        "AI 산업의 최신 동향은 무엇인가요?",
        "오픈AI의 챗GPT 에이전트 기능에 대해 설명해주세요.",
        "구글 딥마인드의 최근 연구 성과는?",
        "미국의 AI 정책 현황은 어떻게 되나요?",
        "AI가 노동 시장에 미치는 영향은?",
        "생성형 AI의 주요 기업들은?",
        "한국의 AI 관련 정책은?",
        "AI 파인튜닝이란 무엇인가요?"
    ]

def export_history():
    """대화 기록을 JSON 형식으로 내보내기"""
    if not conversation_history:
        return "대화 기록이 없습니다."

    return json.dumps(conversation_history, ensure_ascii=False, indent=2)

def clear_history():
    """대화 기록 초기화"""
    global conversation_history
    conversation_history = []
    return "대화 기록이 초기화되었습니다.", "", ""

# ====================================
# 3. Gradio 인터페이스 구성
# ====================================

# CSS 스타일링
custom_css = """
.gradio-container {
    font-family: 'Pretendard', -apple-system, BlinkMacSystemFont, system-ui, Roboto, sans-serif;
}
.contain {
    max-width: 1200px !important;
}
"""

# Gradio 인터페이스 생성
with gr.Blocks(title="SPRi AI Brief 질의응답 시스템", theme=gr.themes.Soft(), css=custom_css) as demo:

    # 헤더
    gr.Markdown("""
    # 🤖 SPRi AI Brief 질의응답 시스템

    학습된 문서를 기반으로 AI가 질문에 답변합니다.
    파인튜닝된 모델과 RAG(Retrieval-Augmented Generation) 기술을 활용합니다.
    """)

    # 메인 인터페이스
    with gr.Row():
        # 왼쪽 패널 - 입력
        with gr.Column(scale=1):
            gr.Markdown("### 📝 질문 입력")

            question_input = gr.Textbox(
                label="질문",
                placeholder="AI 관련 질문을 입력하세요...",
                lines=4
            )

            # 파라미터 설정
            with gr.Accordion("⚙️ 고급 설정", open=False):
                temperature_slider = gr.Slider(
                    minimum=0.1,
                    maximum=1.0,
                    value=0.7,
                    step=0.1,
                    label="Temperature (창의성)"
                )

                max_tokens_slider = gr.Slider(
                    minimum=128,
                    maximum=1024,
                    value=512,
                    step=64,
                    label="최대 토큰 수"
                )

                use_context_checkbox = gr.Checkbox(
                    value=True,
                    label="문서 컨텍스트 사용"
                )

            # 버튼들
            with gr.Row():
                submit_btn = gr.Button("🚀 답변 생성", variant="primary", scale=2)
                clear_btn = gr.Button("🗑️ 초기화", scale=1)

            # 예시 질문
            gr.Markdown("### 💭 예시 질문")
            example_questions = gr.Dropdown(
                choices=get_sample_questions(),
                label="선택하세요",
                interactive=True
            )

        # 오른쪽 패널 - 출력
        with gr.Column(scale=2):
            # 답변 영역
            answer_output = gr.Markdown(
                label="답변",
                value="질문을 입력하고 '답변 생성' 버튼을 클릭하세요."
            )

            # 컨텍스트 표시
            with gr.Accordion("📖 참고 문서", open=False):
                context_output = gr.Markdown(
                    label="검색된 컨텍스트"
                )

            # 통계 정보
            with gr.Accordion("📊 세션 통계", open=False):
                stats_output = gr.Textbox(
                    label="통계",
                    interactive=False
                )

    # 대화 기록 탭
    with gr.Tab("📜 대화 기록"):
        history_output = gr.Textbox(
            label="대화 기록 (JSON)",
            lines=10,
            interactive=False
        )

        with gr.Row():
            export_btn = gr.Button("💾 내보내기")
            refresh_btn = gr.Button("🔄 새로고침")

    # 정보 탭
    with gr.Tab("ℹ️ 정보"):
        gr.Markdown("""
        ### 시스템 정보

        - **모델**: KoAlpaca-Polyglot-5.8B (파인튜닝됨)
        - **임베딩**: ko-sroberta-multitask
        - **벡터 DB**: ChromaDB
        - **청크 크기**: 500 토큰
        - **검색 문서 수**: 3개

        ### 사용 팁

        1. 구체적인 질문일수록 정확한 답변을 받을 수 있습니다
        2. Temperature를 낮추면 일관된 답변, 높이면 창의적인 답변을 얻습니다
        3. 컨텍스트를 사용하면 문서 기반 답변, 사용하지 않으면 모델의 일반 지식 활용
        """)

    # 이벤트 핸들러 연결
    submit_btn.click(
        fn=process_question,
        inputs=[question_input, temperature_slider, max_tokens_slider, use_context_checkbox],
        outputs=[answer_output, context_output, stats_output]
    )

    clear_btn.click(
        fn=clear_history,
        outputs=[answer_output, context_output, stats_output]
    )

    # 예시 질문 선택 시 입력창에 자동 입력
    example_questions.change(
        fn=lambda x: x,
        inputs=[example_questions],
        outputs=[question_input]
    )

    # 엔터키로 제출
    question_input.submit(
        fn=process_question,
        inputs=[question_input, temperature_slider, max_tokens_slider, use_context_checkbox],
        outputs=[answer_output, context_output, stats_output]
    )

    # 대화 기록 관련
    export_btn.click(
        fn=export_history,
        outputs=[history_output]
    )

    refresh_btn.click(
        fn=export_history,
        outputs=[history_output]
    )

# ====================================
# 4. Gradio 실행
# ====================================

# Colab에서 실행
print("🚀 Gradio 인터페이스를 시작합니다...")
demo.launch(
    share=True,  # 외부 접속 가능한 링크 생성
    debug=False,
    height=800
)

print("""
✅ Gradio 인터페이스가 성공적으로 실행되었습니다!

📌 사용 방법:
1. 질문을 입력하거나 예시 질문을 선택하세요
2. '답변 생성' 버튼을 클릭하거나 Enter를 누르세요
3. 고급 설정에서 파라미터를 조정할 수 있습니다
4. 대화 기록은 JSON 형식으로 내보낼 수 있습니다

🔗 공유 링크를 통해 다른 사용자도 접속할 수 있습니다.
""")

✅ 기존 rag_system을 사용합니다.
🚀 Gradio 인터페이스를 시작합니다...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11cdd8b3e94d19c6ac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



✅ Gradio 인터페이스가 성공적으로 실행되었습니다!

📌 사용 방법:
1. 질문을 입력하거나 예시 질문을 선택하세요
2. '답변 생성' 버튼을 클릭하거나 Enter를 누르세요
3. 고급 설정에서 파라미터를 조정할 수 있습니다
4. 대화 기록은 JSON 형식으로 내보낼 수 있습니다

🔗 공유 링크를 통해 다른 사용자도 접속할 수 있습니다.

